In [124]:
from sqlalchemy import create_engine
from urllib.parse import quote_plus

usuario = 'postgres'
senha = quote_plus('@manaus')
host = 'localhost'
porta = '5432'
nome_do_banco = 'db_mrp'

### Carregar o arquivo Excel

In [125]:
import pandas as pd

# Carregar a planilha

file_path = r'arquivo_convertido.xlsx'
xls = pd.ExcelFile(file_path)

# Carregar a planilha específica
df_n_fp = xls.parse('Sheet1', header=0)

# Exibir as primeiras linhas
df_n_fp.head()

,Plant,Target ID,Line,Update,Demand ID,Model.Suffix,Model,Suffix,Tool,BOM Date,...,Original Due Date,MERC PO NO,MERC PO Qty,MERC Supplier Name,Mix Member,Mix Group,Mix Rate,OQC Pass,Sys.Cancel,RNUM
0,NW1,4M2449ST-JOA-20241122002944-0001,A01,NaN,4M4ML001,43UT8000PSA.AWZFLJZ,43UT8000PSA,AWZFLJZ,UP80_21Y,45349,...,45629,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NW1,FO2450ST-JOA-20241128035016-0001,A01,NaN,4M2L00BN,50UT801C0SA.BWZJLJZ,50UT801C0SA,BWZJLJZ,NaN,45502,...,45629,NaN,NaN,NaN,NaN,NaN,NaN,56.0,NaN,NaN
2,NW1,FO2450ST-PTG-20241119221700-0010,A01,NaN,4M2L0080,50UT801C0SA.BWZJLJZ,50UT801C0SA,BWZJLJZ,NaN,45502,...,45628,NaN,NaN,NaN,NaN,NaN,NaN,500.0,NaN,NaN
3,NW1,FO2450ST-PTG-20241119221700-0011,A01,NaN,4M2L0081,50UT801C0SA.BWZJLJZ,50UT801C0SA,BWZJLJZ,NaN,45502,...,45628,NaN,NaN,NaN,NaN,NaN,NaN,419.0,NaN,NaN
4,NW1,FO2450ST-PTG-20241119221700-0020,A01,NaN,4M2L008A,50UT801C0SA.BWZJLJZ,50UT801C0SA,BWZJLJZ,NaN,45502,...,45628,NaN,NaN,NaN,NaN,NaN,NaN,148.0,NaN,NaN


In [126]:
# Identifique as colunas que provavelmente são datas
# Aqui estamos assumindo que as colunas que têm nomes numéricos podem ser convertidas
date_columns = [col for col in df_n_fp.columns if isinstance(col, (int, float))]

# Converta os nomes das colunas para datas
new_column_names = {
    col: pd.to_datetime(col, unit='D', origin='1899-12-30').strftime('%Y-%m-%d')
    for col in date_columns
}

# Renomear as colunas no DataFrame
df_n_fp.rename(columns=new_column_names, inplace=True)

# Exiba as colunas com os novos nomes
print(df_n_fp.head())


  Plant                         Target ID Line Update Demand ID  \
0   NW1  4M2449ST-JOA-20241122002944-0001  A01    NaN  4M4ML001   
1   NW1  FO2450ST-JOA-20241128035016-0001  A01    NaN  4M2L00BN   
2   NW1  FO2450ST-PTG-20241119221700-0010  A01    NaN  4M2L0080   
3   NW1  FO2450ST-PTG-20241119221700-0011  A01    NaN  4M2L0081   
4   NW1  FO2450ST-PTG-20241119221700-0020  A01    NaN  4M2L008A   

          Model.Suffix        Model   Suffix      Tool  BOM Date  ...  \
0  43UT8000PSA.AWZFLJZ  43UT8000PSA  AWZFLJZ  UP80_21Y     45349  ...   
1  50UT801C0SA.BWZJLJZ  50UT801C0SA  BWZJLJZ       NaN     45502  ...   
2  50UT801C0SA.BWZJLJZ  50UT801C0SA  BWZJLJZ       NaN     45502  ...   
3  50UT801C0SA.BWZJLJZ  50UT801C0SA  BWZJLJZ       NaN     45502  ...   
4  50UT801C0SA.BWZJLJZ  50UT801C0SA  BWZJLJZ       NaN     45502  ...   

  Original Due Date MERC PO NO  MERC PO Qty  MERC Supplier Name  Mix Member  \
0             45629        NaN          NaN                 NaN         NaN   


In [127]:
df_n_fp.columns

Index(['Plant', 'Target ID', 'Line', 'Update', 'Demand ID', 'Model.Suffix',
       'Model', 'Suffix', 'Tool', 'BOM Date',
       ...
       'Original Due Date', 'MERC PO NO', 'MERC PO Qty', 'MERC Supplier Name',
       'Mix Member', 'Mix Group', 'Mix Rate', 'OQC Pass', 'Sys.Cancel',
       'RNUM'],
      dtype='object', length=262)

In [128]:
# remover colunas nao usadas
df_n_fp = df_n_fp.drop(columns=['Target ID', 'Line', 'Update', 'Demand ID', 'Model', 'Suffix', 'Tool', 'BOM Date', 'Original Due Date', 'MERC PO NO', 'MERC PO Qty', 'MERC Supplier Name', 'Mix Member', 'Mix Group', 'Mix Rate', 'OQC Pass', 'Sys.Cancel', 'RNUM', 'Demand Type', 'Source Type', 'Map Qty', 'RSD', 'Due Date', 'PST', 'Ship To Name', 'Bucket', 'Urgent Back Order','Mapping Order', 'Ship Method', 'Mapping Qty', 'Mapping Date','PST-Due Date', 'GSBS Doc. Due Date', 'Booking ID', 'Vessel Name', 'Ship Alloc. Initial POL ETD', 'Lot Qty',   'Remain Qty', 'Comment1', 'Comment2', 'CBM', 'Market', 'Product Type', 'PL2 Code', 'PL4 Code', 'Chassis', 'Country', 'Expire Date', 'ATP Date','Back Order', 'Final Dest.', 'Closed Qty', 'Parent Closed Qty', 'Result Qty', 'Period Qty', 'Hold Date', 'SMS', 'Item Status','UIT', 'Set Pno', 'UPH', 'Ori. Demand ID', 'Splitted Demand ID', 'Ship To Code', 'W/O', 'W/O Line',       'W/O PST', 'W/O PET', 'W/O Complete', 'Bill To Name', 'Long-term Prod(M6)', 'BOM', 'New', 'Priority', 'PET', 'W/O Status', 'Latest T/T'])

In [129]:
print(df_n_fp.columns)


Index(['Plant', 'Model.Suffix', '2024-12-01', '2024-12-02', '2024-12-03',
       '2024-12-04', '2024-12-05', '2024-12-06', '2024-12-07', '2024-12-08',
       ...
       '2024-05-23', '2024-05-24', '2024-05-25', '2024-05-26', '2024-05-27',
       '2024-05-28', '2024-05-29', '2024-05-30', '2024-05-31', '2024-06-01'],
      dtype='object', length=185)


In [130]:
df_n_fp.head()

,Plant,Model.Suffix,2024-12-01,2024-12-02,2024-12-03,2024-12-04,2024-12-05,2024-12-06,2024-12-07,2024-12-08,...,2024-05-23,2024-05-24,2024-05-25,2024-05-26,2024-05-27,2024-05-28,2024-05-29,2024-05-30,2024-05-31,2024-06-01
0,NW1,43UT8000PSA.AWZFLJZ,NaN,NaN,20.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NW1,50UT801C0SA.BWZJLJZ,NaN,NaN,56.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NW1,50UT801C0SA.BWZJLJZ,NaN,NaN,500.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NW1,50UT801C0SA.BWZJLJZ,NaN,NaN,419.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NW1,50UT801C0SA.BWZJLJZ,NaN,NaN,148.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [131]:
print(df_n_fp.columns)
print(df_n_fp.dtypes)


Index(['Plant', 'Model.Suffix', '2024-12-01', '2024-12-02', '2024-12-03',
       '2024-12-04', '2024-12-05', '2024-12-06', '2024-12-07', '2024-12-08',
       ...
       '2024-05-23', '2024-05-24', '2024-05-25', '2024-05-26', '2024-05-27',
       '2024-05-28', '2024-05-29', '2024-05-30', '2024-05-31', '2024-06-01'],
      dtype='object', length=185)
Plant            object
Model.Suffix     object
2024-12-01      float64
2024-12-02      float64
2024-12-03      float64
                 ...   
2024-05-28      float64
2024-05-29      float64
2024-05-30      float64
2024-05-31      float64
2024-06-01      float64
Length: 185, dtype: object


In [132]:
df_n_fp.head()

,Plant,Model.Suffix,2024-12-01,2024-12-02,2024-12-03,2024-12-04,2024-12-05,2024-12-06,2024-12-07,2024-12-08,...,2024-05-23,2024-05-24,2024-05-25,2024-05-26,2024-05-27,2024-05-28,2024-05-29,2024-05-30,2024-05-31,2024-06-01
0,NW1,43UT8000PSA.AWZFLJZ,NaN,NaN,20.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NW1,50UT801C0SA.BWZJLJZ,NaN,NaN,56.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NW1,50UT801C0SA.BWZJLJZ,NaN,NaN,500.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NW1,50UT801C0SA.BWZJLJZ,NaN,NaN,419.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NW1,50UT801C0SA.BWZJLJZ,NaN,NaN,148.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [133]:
# Supondo que 'Plant' e 'Model.Suffix' sejam colunas fixas e as demais sejam datas
id_vars = ["Plant", "Model.Suffix"]  # Colunas que permanecerão fixas
value_vars = [col for col in df_n_fp.columns if col not in id_vars]  # Colunas que serão derretidas

# Derreter as colunas de data
df_n_fp = pd.melt(
    df_n_fp,
    id_vars=id_vars,
    value_vars=value_vars,
    var_name="date",    # Nome da coluna que armazenará as datas
    value_name="quantity"  # Nome da coluna que armazenará os valores
)

# Converter a coluna "date" para o tipo datetime, se necessário
df_n_fp["date"] = pd.to_datetime(df_n_fp["date"])

# Exibir o DataFrame final
print(df_n_fp.head())


  Plant         Model.Suffix       date  quantity
0   NW1  43UT8000PSA.AWZFLJZ 2024-12-01       NaN
1   NW1  50UT801C0SA.BWZJLJZ 2024-12-01       NaN
2   NW1  50UT801C0SA.BWZJLJZ 2024-12-01       NaN
3   NW1  50UT801C0SA.BWZJLJZ 2024-12-01       NaN
4   NW1  50UT801C0SA.BWZJLJZ 2024-12-01       NaN


In [134]:
df_n_fp.head(80)

,Plant,Model.Suffix,date,quantity
0,NW1,43UT8000PSA.AWZFLJZ,2024-12-01,NaN
1,NW1,50UT801C0SA.BWZJLJZ,2024-12-01,NaN
2,NW1,50UT801C0SA.BWZJLJZ,2024-12-01,NaN
3,NW1,50UT801C0SA.BWZJLJZ,2024-12-01,NaN
4,NW1,50UT801C0SA.BWZJLJZ,2024-12-01,NaN
...,...,...,...,...
75,NW1,43UR781C0SA.AWZFLJZ,2024-12-01,NaN
76,NW1,50UT8050PSA.AWZJLJZ,2024-12-01,NaN
77,NW1,50UT8050PSA.AWZJLJZ,2024-12-01,NaN
78,NW1,50UT8050PSA.AWZJLJZ,2024-12-01,NaN


In [144]:
from sqlalchemy.types import Integer
# Configuração da conexão com o PostgreSQL
engine = create_engine(f'postgresql+psycopg2://{usuario}:{senha}@{host}:{porta}/{nome_do_banco}')

# Exportar o DataFrame para o banco de dados PostgreSQL
df_n_fp.to_sql('table_n_fp_teste', con=engine, if_exists='replace', index=False, dtype={'quantity': Integer()})
print('Dados salvos com sucesso no banco de dados PostgreSQL.')

Dados salvos com sucesso no banco de dados PostgreSQL.


In [136]:
from datetime import datetime
import os

def extrair_data(file_path):
    data_modificacao = os.path.getmtime(file_path)
    data_arquivo = datetime.fromtimestamp(data_modificacao).strftime('%Y-%m-%d')
    return data_arquivo

data_arquivo = extrair_data(file_path)
print(data_arquivo)

2025-01-02


In [137]:
df_n_fp['date_file'] = data_arquivo

In [138]:
df_n_fp.head()

,Plant,Model.Suffix,date,quantity,date_file
0,NW1,43UT8000PSA.AWZFLJZ,2024-12-01,NaN,2025-01-02
1,NW1,50UT801C0SA.BWZJLJZ,2024-12-01,NaN,2025-01-02
2,NW1,50UT801C0SA.BWZJLJZ,2024-12-01,NaN,2025-01-02
3,NW1,50UT801C0SA.BWZJLJZ,2024-12-01,NaN,2025-01-02
4,NW1,50UT801C0SA.BWZJLJZ,2024-12-01,NaN,2025-01-02


In [139]:
# Supondo que você tenha o DataFrame df e deseja mover 'date_file' para a primeira posição
colunas = ['date_file'] + [col for col in df_n_fp.columns if col != 'date_file']

# Reindexando para tornar 'date_file' a primeira coluna
df_n_fp = df_n_fp[colunas]

# Exibir o DataFrame reordenado
print(df_n_fp)


         date_file Plant         Model.Suffix       date  quantity
0       2025-01-02   NW1  43UT8000PSA.AWZFLJZ 2024-12-01       NaN
1       2025-01-02   NW1  50UT801C0SA.BWZJLJZ 2024-12-01       NaN
2       2025-01-02   NW1  50UT801C0SA.BWZJLJZ 2024-12-01       NaN
3       2025-01-02   NW1  50UT801C0SA.BWZJLJZ 2024-12-01       NaN
4       2025-01-02   NW1  50UT801C0SA.BWZJLJZ 2024-12-01       NaN
...            ...   ...                  ...        ...       ...
958915  2025-01-02   NWV  IL7SB-8G4JC.ASAKK01 2024-06-01       NaN
958916  2025-01-02   NWV  IL7SB-8G4JC.ASAKK01 2024-06-01       NaN
958917  2025-01-02   NWV  IL7SB-8G4JC.ASAKJ01 2024-06-01       NaN
958918  2025-01-02   NWV  IL7SB-8G4JC.ASAKK01 2024-06-01       NaN
958919  2025-01-02   NWV  IL7SB-8G4JC.ASAKK01 2024-06-01       NaN

[958920 rows x 5 columns]


In [141]:
import hashlib

def generate_hash(temp_id):
    """
    Gera um hash único com base em campos específicos.
    """
    hash_input = f"{temp_id}"
    return hashlib.md5(hash_input.encode()).hexdigest()

In [142]:
# Passo 1: Criar uma coluna 'temp_id' no DataFrame com números crescentes
df_n_fp['temp_id'] = range(1, len(df_n_fp) + 1)

# Passo 2: Gerar o hash baseado na combinação das colunas
df_n_fp['hash_id'] = df_n_fp.apply(
        lambda row: generate_hash(row['temp_id']),
    axis=1
)


In [143]:
df_n_fp.head()

,date_file,Plant,Model.Suffix,date,quantity,temp_id,hash_id
0,2025-01-02,NW1,43UT8000PSA.AWZFLJZ,2024-12-01,NaN,1,c4ca4238a0b923820dcc509a6f75849b
1,2025-01-02,NW1,50UT801C0SA.BWZJLJZ,2024-12-01,NaN,2,c81e728d9d4c2f636f067f89cc14862c
2,2025-01-02,NW1,50UT801C0SA.BWZJLJZ,2024-12-01,NaN,3,eccbc87e4b5ce2fe28308fd9f2a7baf3
3,2025-01-02,NW1,50UT801C0SA.BWZJLJZ,2024-12-01,NaN,4,a87ff679a2f3e71d9181a67b7542122c
4,2025-01-02,NW1,50UT801C0SA.BWZJLJZ,2024-12-01,NaN,5,e4da3b7fbbce2345d7772b0674a318d5
